In [1]:
#imports for data processing
import nltk
from nltk.stem.lancaster import LancasterStemmer
import numpy as np
import tflearn
import tensorflow as tf
import unicodedata
import sys
from tqdm import tqdm
import pandas as pd
import os
from random import shuffle
import string
import gensim

class LabeledLineSentence():
    def __init__(self, doc_list, label,categories):
        self.label = label     
        self.doc_list = doc_list
        self.categories = categories
    def __iter__(self):
        for idx, doc in enumerate(self.doc_list):
            #print(self.label[idx][0])
            #idx = self.labels.index()
            yield gensim.models.doc2vec.LabeledSentence(doc,    
[self.categories[self.label[idx][0]]])
            
class SetData():
    def __init__(self):
        self.num_classes = 0
        self.classes = []
        self.train_data = None
        self.test_data = None
        self.train_data_dir = '/Users/arunprakash/Documents/TensorFlow/MovieData/train'
        self.test_data_dir = '/Users/arunprakash/Documents/TensorFlow/MovieData/test'
        for _, dirnames, _ in os.walk(self.train_data_dir):
            self.num_classes += len(dirnames)
            if len(dirnames)>0:
                self.classes = dirnames
        self.stemmer = LancasterStemmer()
        self.tbl = str.maketrans({key: None for key in string.punctuation})

    def one_hot_classes(self,clsname):
        ohe = []
        for i in self.classes:
            if i == clsname:
                ohe.append(1)
            else:
                ohe.append(0)
        return ohe

    def remove_punctuation(self,text):
        return text.translate(self.tbl)


    def train_data_with_label(self):
        train_text = []
        tr_words = []
        for cls in self.classes:
            train_data_path = self.train_data_dir+ '/' + cls
            for i in tqdm(os.listdir(train_data_path)):
                if i != '.DS_Store':
                    path = os.path.join(train_data_path, i)
                    df = pd.read_csv(path,delimiter='\n')
                    dfnp = df.values
                    fa = dfnp.flatten('F')
                    li = ''.join(fa.tolist())
                    dlg = self.remove_punctuation(li)
                    dlg_tkn = nltk.word_tokenize(dlg)
                    train_text.append([dlg_tkn, [self.classes.index(cls)]])
        fd = np.array(train_text)
        #np.save('train_text_part.npy', train_text)
        #np.save('train_text_wrds_part.npy', tr_words)
        return fd

    def get_vec_for_new_data(self,path):
        test_text = []
        df = pd.read_csv(path,delimiter='\n')
        dfnp = df.values
        fa = dfnp.flatten('F')
        li = ''.join(fa.tolist())
        dlg = self.remove_punctuation(li)
        dlg_tkn = nltk.word_tokenize(dlg)
        test_text.append(dlg_tkn)
        return test_text

/anaconda3/envs/tfdeeplearningv1.4/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)
/anaconda3/envs/tfdeeplearningv1.4/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
sd = SetData()

In [3]:
fd_text = sd.train_data_with_label();   
doc = fd_text[:,0]
lb = fd_text[:,1]
it = LabeledLineSentence(doc, lb,sd.classes)
model = gensim.models.Doc2Vec(size=300, min_count=0, alpha=0.025, min_alpha=0.025)
model.build_vocab(it)
model.train(it, epochs=100, total_examples=model.corpus_count)

100%|██████████| 21/21 [00:00<00:00, 39.77it/s]
/anaconda3/envs/tfdeeplearningv1.4/lib/python3.5/site-packages/gensim/models/doc2vec.py:366: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
/anaconda3/envs/tfdeeplearningv1.4/lib/python3.5/site-packages/ipykernel_launcher.py:32: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).


In [4]:
for cls in sd.classes:
    test_data_path = sd.test_data_dir+ '/' + cls
    print(cls+': ')
    for i in tqdm(os.listdir(test_data_path)):
        if i != '.DS_Store':
            path = os.path.join(test_data_path, i)
            td = sd.get_vec_for_new_data(path)
            v1 = model.infer_vector(td[0])
            print(model.docvecs.most_similar([v1])[0][0], model.docvecs.most_similar([v1])[1][0])

 17%|█▋        | 1/6 [00:00<00:00,  6.53it/s]

comedy: 
comedy sciencefic


 67%|██████▋   | 4/6 [00:00<00:00, 10.82it/s]

drama comedy
comedy drama


  0%|          | 0/4 [00:00<?, ?it/s]

...

drama action
sciencefic action
drama: 
drama action


 67%|██████▋   | 4/6 [00:00<00:00,  8.89it/s]

drama action
comedy action


100%|██████████| 6/6 [00:00<00:00,  7.51it/s]

action comedy
drama sciencefic


In [11]:
res_var = ''
for cls in sd.classes:
    test_data_path = sd.test_data_dir+ '/' + cls
    res_var += cls +': \n'
    for i in tqdm(os.listdir(test_data_path)):
        if i != '.DS_Store':
            path = os.path.join(test_data_path, i)
            td = sd.get_vec_for_new_data(path)
            v1 = model.infer_vector(td[0])
            res_tup = model.docvecs.most_similar([v1])[0]
            res_var += res_tup[0] + '--->' + str(res_tup[1]) + '\t\t\t'
            res_tup = model.docvecs.most_similar([v1])[1]
            res_var += res_tup[0] + '--->' + str(res_tup[1]) + '\n'
print(res_var)

100%|██████████| 6/6 [00:00<00:00,  9.75it/s]

comedy: 
comedy--->0.1808563768863678			action--->0.1149946004152298
drama--->0.09855122119188309			comedy--->0.07259517908096313
comedy--->0.1209990382194519			action--->0.07334736734628677
comedy--->0.16781514883041382			action--->0.09096236526966095
comedy--->0.15442846715450287			action--->0.1516314148902893
sciencefic: 
comedy--->0.1113196611404419			sciencefic--->0.09614986926317215
horror--->0.13223889470100403			action--->0.13110636174678802
sciencefic--->0.21599653363227844			action--->0.1879524141550064
horror: 
comedy--->0.23270288109779358			horror--->0.15649518370628357
sciencefic--->0.08977347612380981			comedy--->0.05700945109128952
comedy--->0.166287362575531			horror--->0.16544024646282196
horror--->0.9132819175720215			action--->0.646616518497467
horror--->0.16922016441822052			drama--->0.11785160005092621
horror--->0.18545782566070557			sciencefic--->0.09572823345661163
action: 
action--->0.0634859949350357			comedy--->0.051785193383693695
comedy--->0.142584860324859